# 3 함수, 반복자, 제네레이터
함수형 프로그래밍의 핵심은 순수 함수를 사용해 입력 정의역(domain)의 값을 출력 치역(range) 값으로 바꾸는 것이다. 

순수 함수에는 부수 효과(외부의 상태를 변경 또는 함수로 들어온 인자의 상태를 직접 변경하는 것)가 없으며 따라서 변수 의존 관계를 줄일 수 있다. 파이썬에서 대입문을 완전히 없앨 수는 없겠지만, 상태가 있는 객체에 의존하지 않을 수 있다. 이를 위해서 파이썬이 제공하는 내장 데이터 구조 중 상태가 있는 연산을 필요로 하지 않는 구조를 활용해야 한다.

따라서 이번 장은 파이썬의 기능 몇 가지를 살펴볼 것이다.

* 부수 효과가 없는 순수 함수

* 인자로 넘길 수 있거나 함수의 결과로 반환할 수 있는 객체인 함수

* 객체지향적인 후위 표기법이나 전위 표기법으로 파이썬 문자열 사용

* tuple이나 namedtuple을 사용해 상태가 없는 객체 생성

* 반복 가능한 컬렉션을 함수형 프로그래밍을 위한 주 설계 도구로 활용

2장에서 살폈듯 파이썬은 재귀 깊이에 제한을 두며, 자동으로 TCO를 진행하지 않는다. 따라서 제너레이터 식을 이용해 직접 재귀를 최적화해야 한다. 여기서는 다음과 같은 작업을 수행하는 제너레이터 식을 작성할 것이다.

* 변환

* 재구성 

* 복잡한 계산


---


* 객체의 상태: 상태는 **특정 시점에 객체가 가지고 있는 정보의 집합**으로 객체의 구조적 특징을 표현한다. 객체의 상태는 객체에 존재하는 정적인 프로퍼티와 동적인 프로퍼티 값으로 구성된다. 객체의 프로퍼티는 단순한 값과 다른 객체를 참조하는 링크로 구분할 수 있다.


---

## 순수 함수 작성하기
순수 함수는 부수 효과가 없다. 즉, 변수의 전역적인 상태를 변경하는 일이 결코 없다. global문을 사용하지 않는다면 이러한 목표를 거의 충족할 수 있을 것이다. 또한 상태를 바꿀 수 있는 객체를 다루는 방식을 바꿔야만 한다. 자유 변수(free variable)를 사용해 파이썬 전역에 있는 값을 참조하면 매개변수를 적절히 사용해 처리할 수 있다.

다음은 전역적인 문장의 사용 예를 보여주는 예제다.

In [1]:
def some_function(a, b, t):
    
    return a+b+t+global_adjustment

위 예제에서 함수를 리펙토링하여 global_adjustment 변수를 적절한 매개변수로 바꿀 수 있다. 하지만 이런 작업은 복잡한 애플리케이션에서 많은 파장을 불러일으킬 수 있다.

상태가 있는 파이썬 내부 객체는 많다. file 클래스의 인스턴스나 모든 file-like 객체가 자주 사용되는 '상태가 있는 객체'다. 상태가 있는 파일 객체를 잘 정의된 영역 안으로 제한할 수 있는 **with문**을 잘 활용하는 것이 중요하다.(파일 객체는 항상 with 컨텍스트 안에서 사용하라)

전역 파일 객체, 전역 데이터베이스 연결 등을 피하고 그와 관련 있는 상태를 피해야 한다. 전역 파일 객체는 열린 파일을 차리하는 경우에 매우 흔하게 사용하는 패턴이다. 다음 코드를 보자.

In [ ]:
def open_(iname, oname):
    
    global ifile, ofile
    
    ifile = open(iname, "r")
    ofile = open(onamem "w")

위 코드는 다양한 다른 함수들이 ifile이나 ofile 변수를 사용할 수 있도록 열어두었다. 이는 좋은 설계가 아니며 피해야 한다. 파일은 함수에 제공되는 매개변수여야 하며, 열린 파일은 with문으로 감싸서 상태에 따른 동작을 제대로 처리하도록 설계해야 한다.


---

## first class 객체인 함수
함수가 속성(attribute)가 있는 객체이기 때문에 우리는 __name__이나 __name__ 속성을 사용해 docstring이나 함수의 이름을 뽑아낼 수 있다. 또한 함수의 본문을 __code__ 속성을 이용해 가져올 수도 있다. 

함수를 변수에 대입하거나, 함수를 인자로 넘기거나, 함수를 값으로 반환할 수도 있다. 이러한 기법을 이용해 고차 함수를 쉽게 만들 수도 있다.

이와 아울러 호출 가능한 객체를 사용해 함수를 정의할 수도 있다. 호출 가능 객체 역시 first class 객체다. 심지어 호출 가능 클래스 정의를 고차 함수로 볼 수도 있다. 다만 호출 가능 객체에서 __init()__ 메서드를 사용하는 방식은 신중하게 생각해야 한다. 한 가지 일반적인 응용 방법은 __init()__ 메서드를 사용해 전략 디자인 패턴(Strategy design pattern)에 부합하는 객체를 만드는 것이다. 

전략 디자인 패턴을 따르는 클래스는 알고리즘이나 알고리즘 일부를 제공하는 다른 객체에 의존한다. 이 패턴은 알고리즘의 자세한 부분을 클래스 안에 컴파일해 넣는 대신, 실행 시점에 알고리즘의 세부 사항을 주입할 수 있게 해준다.

아래는 내장된 전략 객체가 있는 호출 가능한 객체 예제다.

In [2]:
import collections

class Mersennel(collections.Callable):
    
    def __init__(self, algorithm):
        
        # 주어진 인자만큼 2를 거듭제곱하는 함수를 참조
        self.pow2 = algorithm
        
    def __call__(self, arg):
        
        # 인자를 받아서 함수를 적용한 뒤 나온 값에 1을 뺀 값을 반환
        return self.pow2(arg) - 1

/var/folders/p3/fjkwnjwd4dzcfcqt47dlgrm40000gn/T/ipykernel_14196/7786008.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class Mersennel(collections.Callable):


이 클래스는 __init__()를 사용해 다른 함수 참조를 저장한다. 하지만 아무런 상태가 있는 인스턴스 변수를 만들지 않는다. 이 클래스에 끼워 넣을 수 있는 세 가지 후보 객체는 다음과 같다.

In [4]:
# 주어진 인자만큼 2를 거듭제곱하는 함수(b: 지수)


# 비트 left shift 연산으로 2의 거듭제곱을 계산한다.
def shifty(b):
    
    # 비트 1을 왼쪽으로 shift할 떄마다 2의 거듭제곱만큼 값이 커지는 것
    return 1 << b


# 재귀를 이용해 2의 거듭제곱을 계산한다. 인자로 받은 값에서 -1씩 줄어들며 0이 되면 끝난다.
def multy(b):
    
    # 기저 조건
    if b == 0:
        
        return 1
    
    # 2*multy(b-1)=2*2*multy(b-2)=...=2^b
    return 2*multy(b-1)


# 재귀함수
# faster 함수는 분할 정복(divide and conquer) 전략을 사용해 b번이 아니라 log_2(b)번의 곱셈을 수행한다.(시간 효율성)
# 과정에서 b를 계속 2씩 나누는 대신, t*t로 지수는 *2가 되어 보상된다.
def faster(b):
    
    # 기저 조건
    if b == 0:
        
        return 1
    
    # b가 홀수면
    if b % 2 == 1:
        
        # 홀수 재귀
        # 홀수 턴에서는 2^1만큼 늘어난다.
        return 2*faster(b-1)
    
    # b가 짝수면
    # 짝수 재귀(log_2(b)번의 곱셈을 수행하게 만드는 핵심)
    # t: b를 2로 나눈 몫을 faster() 함수 인자로 넣어서 실행한다.
    # 짝수는 2로 나눠주는 만큼, 반환값도 ()^2로 지수가 2배로 늘어난다.
    t = faster(b//2)
    
    return t*t

알고리즘 전략을 내장한 Mersennel 클래스의 인스턴스는 다음과 같이 만들 수 있다.

In [5]:
# 첫 번째 객체
mls= Mersennel(shifty)

# 두 번째 객체
mlm= Mersennel(multy)

# 세 번째 객체
mlf= Mersennel(faster)

---

## 문자열 사용하기
파이썬 문자열은 변경 불가능하기 때문에 함수형 프로그래밍 객체의 좋은 예라고 할 수 있다. 파이썬의 string 모듈에는 많은 메서드가 들어 있고, 그들 모두는 결과로 새로운 문자열을 내놓는다. 이러한 메서드는 부수 효과가 없는 순수 함수다.

전위 방식인 대부분의 함수와는 달리 string 메서드 함수를 사용하는 구문들은 후위 방식이다. 때문에 복잡한 문자열 연산이 일반적인 함수와 혼합이 되는 일이 발생하면 가독성이 떨어질 수 있다.

아래는 구분 기호를 제거하여 애플리케이션의 다른 곳에서 사용할 수 있는 Decimal 객체를 반환하는 정리 함수다. 이 함수를 구현하면서 전위 연산자와 후위 연산자를 번갈아 쓰기 때문에 가독성이 어떻게 떨어지는지 알 수 있을 것이다.

In [1]:
# 십진법 기반의 연산을 처리하는 decimal 모듈. 내장 클래스 Decimal을 사용할 것이다.
from decimal import *

def clean_decimal(text):
    
    if text is None:
        
        return text
    
    # try 구문 중 에러가 발생하면 except 블록이 수행된다.
    try:
        
        # 문자열에서 $와 ,를 제거해서 반환한다.
        return Decimal(text.replace("$", "").replace(",", ""))
    
    except InvalidOperation:
        
        return text

위 함수는 문자열에서 $와 ,를 제거하기 위해 replace를 두 번 호출했다. 그 결과로 생성한 문자열을 Decimal 클래스 생성자의 인자로 전달하고, 그 결과로 생긴 객체를 반환했다. 

위 예제를 일관성 있게 구현하려면 다음과 같이 string 메서드 함수를 처리하는 전위 연산자 함수를 정의해야 한다.

In [2]:
def replace(data, a, b):
    
    return data.replace(a, b)

위처럼 정의한다면 좀 더 일관성 있게 코드를 구현할 수 있다. 예제의 경우 문자열에서 $와 ,를 제거해서 반환하는 부분을 

> Decimal(replace( replace(text, "$", ""), ",", ""))

라는 구문으로 일관성 있게 작성할 수 있다.

In [3]:
replace = str.replace

replace("$12.45", "$", "")

'12.45'

좀 더 나은 접근 방식은 다음과 같이 비슷하게 구분 기호를 정리해주는 전위 연산자 방식의 함수를 정의하는 것이다.

In [4]:
# 재귀적으로 구분 기호를 제거해 준다.
# chars 변수에는 구분 기호가 있고("$,"), str에서 이 구분 기호를 발견하면 차례로 제거하게 된다.
def remove( str, chars ):
    
    if chars:
        
        # chars 변수의 맨 앞 기호를 제거했다면, chars는 [1:]로 슬라이싱해서 재귀를 적용한다.
        return remove( str.replace(chars[0], ""), chars[1:] )
    
    return str

위를 이용한다면

> Decimal(remove(text, "$,")) 

와 같이 문자열을 정리($와 ,를 문자열에서 제거)하는 목적을 더 잘 드러낼 수 있다.


---

## tuple과 namedtuple 사용하기
파이썬의 tuple(튜플)은 변경 불가능한 객체이므로 마찬가지로 함수형 프로그래밍 객체로 적합하다. tuple은 소수의 메서드 함수만 제공하기 때문에 거의 대부분의 작업은 전휘 문법을 사용한 함수를 이용해 이뤄진다. 주로 튜플의 리스트, 튜플의 튜플, 튜플들을 만들어 내는 제네레이터를 주로 사용한다.

물론, namedtuple(이름 있는 튜플)이라는 튜플의 필수적인 기능도 살핀다. 이는 인덱스 대신 이름을 사용할 수 있게 해준다. namedtuple을 사용하면 데이터를 취합한 객체를 만들 수 있다. tuple이나 namedtuple 중 어느 것을 사용할지는 순전히 편의 문제다.

값의 모음의 경우 대부분 튜플(또는 이름 있는 튜플)을 사용할 것이다. 값을 하나만 사용하거나 단순히 두 값을 필요로 하는 경우에는 보통 함수에 이름이 정해진 매개변수를 전달하는 방식을 사용할 것이다. 하지만 컬렉션을 사용하는 경우에는 튜플의 반복자나 이름 있는 튜플의 반복자를 사용해야 될 수도 있다.

(수, 수, 수) 형태의 RGB 튜플 리스트가 있다고 가정하자. 아래는 3-튜플에서 값을 가져오기 위해 함수를 사용하는 예제다.

In [5]:
red = lambda color: color[0]

green = lambda color: color[1]

blue = lambda color: color[2]

물론 다음과 같이 namedtuple을 사용할 수도 있다.

In [7]:
from collections import namedtuple

Color = namedtuple("Color", ("red", "green", "blue", "name"))

이렇게 하면 red(item) 대신 item.red를 사용할 수 있다.


---

## 제네레이터 식 사용하기
이번엔 몇 가지 복잡한 제네레이터 기법을 살핀다. list나 dict 내장을 통해 list나 dict 리터럴을 만들어 내는 제네레이터 식에서 흔히 사용하는 파이썬 문법 일부가 등장할 것이다. list 디스플레이(display)나 내장(comprehension)은 제네레이터 식을 사용하는 방법의 예다. 리스트 디스플레이에는 [x**2 for x in range(10];과 같은 리터럴 문법이 들어간다. 

제네레이터 식을 사용할 떄 발생할 수 있는 두 가지 문제점은 반드시 알아둬야 한다.

* 제네레이터는 컬렉션 크기를 알아야 할 필요가 있는 len()과 같은 함수인 경우를 제외하고는 리스트와 비슷해 보인다.

* 제네레이터는 오직 한 번만 사용할 수 있다. 일단 사용하고 나면 제네레이터는 비어 있는 것으로 보인다.

앞으로 몇몇 예제에서 사용하게 될 제네레이터 식은 다음과 같다. 소인수(주어진 자연수를 나누어 떨어뜨리는 약수 중에서 소수인 약수)를 구한다.

In [15]:
import math
# 주어진 수의 소인수(prime factor)를 도출한다.
# 2를 제외한 모든 모든 소수는 홀수이기 때문에 2를 따로 처리한다.(반복 횟수를 절반으로 줄일 수 있다.)
def pfactors1(x):
    
    # x가 짝수(2보다 큰 짝수는 소수일 수 없다.)
    if x % 2 ==0:
        
        yield 2
        
        # 2로 나눈 몫이 1보다 크다면, 즉 2보다 큰 짝수면
        if x // 2 > 1:
            
            # 재귀
            # 그 후에는 x / 2의 소인수를 검사한다.
            yield from pfactors1(x//2)
            
        return
    
    
    # 3부터 x 제곱근까지의 값을 홀수 단위로 내놓는다.
    # 작은 값부터 나누고 그 몫을 재귀하기 때문에 결과적으로 소인수만 도출된다.
    # (3으로 나눌 수 있을 때까지 나누고, 5로 나눌 수 있을 때까지 나누고, 7로 나눌 수 있을 때까지 나누고,...)
    # 꼬리재귀 최적화 코드
    for i in range(3, int(math.sqrt(x) + .5), 2):
        
        # x가 i로 나누어지면
        if x % i == 0:
            
            # 인수 i를 내놓는다.
            yield i
            
            # 그 후 재귀적으로 x / i의 소인수를 찾는다.
            if x // i > 1:
                
                # 재귀. 그 몫의 소인수를 검사한다.
                yield from pfactors1(x//i)
                
            return
        
        
    yield x

다만 재귀적인 제네레이터 함수에서는 return문을 사용할 때 조심해야 한다. 다음과 같은 문장은 사용해선 안 된다.

```
return recursive_iter(args)
```

위와 같이 쓰면 제네레이터 객체를 반환하기만 하고, 만들어진 값을 반환하도록 함수를 평가하지는 않는다. 따라서 다음과 같은 방식을 사용하거나

```
for result in recursive_iter(args):

    yield result
```
   
다음과 같은 방식을 사용하라.

```
yield from recursive_iter(args)
```

어쨌든 앞서 만든 코드의 대안으로 더 순수한 재귀를 사용할 수 있다.

In [12]:
def pfactorsr(x):
    
    # 내부 재귀함수
    def factor_n(x, n):
        
        # n의 거듭제곱이 x보다 큰 경우 마지막으로 소수 x를 반환하고 마친다.
        if n * n > x:
            
            yield x
            
            return
        
        # x가 n으로 나누어지면, n을 반환한다.
        if x % n == 0:
            
            yield n
            
            # 만약 x/n의 몫이 1보다 크면
            if x // n > 1:
                
                # 그 몫과 n을 내부 재귀함수로 검사한다.
                yield from factor_n(x//n, n)
                
        else:
            
            # 홀수로 올라간다.
            yield from factor_n(x, n+2)
    
    
    
    # x가 짝수인 경우
    if x % 2 == 0:
        
        # x=2인 경우 2를 반환한다.
        yield 2
        
        # x가 2보다 큰 짝수일 경우
        if x // 2 > 1:
            
            # 그 몫을 재귀적으로 검사한다.
            yield from pfactorsr(x//2)
            
        return
    
    
    # 검사하는 수 x와 3을 내부 재귀함수에 대입한다. 
    yield from factor_n(x, 3)

---

## 제네레이터의 한계
제네레이터 식이나 함수에는 몇 가지 한계가 있다는 것을 지적했다. 다음은 이런 한계를 확인해 볼 것이다.

In [13]:
pfactors1(1560)

<generator object pfactors1 at 0x1195716d0>

첫 번째 예제에서는 제네레이터 식이 엄격하지 않다는 것을 알 수 있다. 지연 계산을 수행하며, 해당 제네레이터 함수를 소비하기 전까지는 적절한 값이 들어있지 않다. 이 자체는 한계가 아니다. 이는 제네레이터가 파이썬을 사용한 함수형 프로그래밍에 잘 들어맞는 이유라고 할 수 있다.

In [16]:
list(pfactors1(1560))

[2, 2, 2, 3, 5, 13]

두 번째 예에서는 제네레이터 식에서 리스트 객체를 구체화했다. 출력을 살피거나 단위 테스트 케이스를 작성하는 경우 이런 기능이 유용하다. 문제는 다음 케이스다.

In [17]:
len(pfactors1(1560))

TypeError: object of type 'generator' has no len()

세 번째 예제에서는 제네레이터 함수의 한 가지 한계인 len()이 없다는 것을 알 수 있다.

제네레이터 함수의 다른 한계점으로는 오직 한 번밖에 쓸 수 없다는 점을 들 수 있다. 아래 예제를 보자

In [18]:
result= pfactors1(1560)

sum(result)

27

In [19]:
sum(result)

0

첫 sum() 메서드 평가는 제네레이터를 모두 평가했다. 하지만 다시 sum()을 평가하면 제네레이터가 비어있음을 알 수 있다. 제네레이터가 만드는 값은 오직 한 번만 소비할 수 있다.

파이썬의 제네레이터는 상태가 있다. 일부 함수형 프로그래밍에 있어서 제네레이터는 유용하지만, 그렇게 완전하지는 않다.

itertools.tee() 메서드를 이용하면 이 한계를 극복할 수 있다.(이는 8장에서 살핀다.) 여기서는 간단히 사용 예제만 살펴보자.

In [ ]:
import itertools

def limits(iterable):
    
    max_tee, min_tee = itertools.tee(iterable, 2)
    
    return max(max_tee), min(min_tee)

매개변수로 받은 제네레이터 식을 복사하여 max_tee()와 min_tee()를 만들었다. 이렇게 하면 원래의 반복자는 그대로 남는다. 이 두 객체를 사용하면 해당 반복 가능 객체를 가지고 최댓값과 최솟값을 얻을 수 있다.


---

## 제네레이터 식 조합하기
함수형 프로그래밍의 핵심은 좀 더 복잡한 복합 처리를 구성하기 위해 제네레이터 식이나 제네레이터 함수를 쉽게 결합할 수 있다는 점에 있다. 제네레이터 식을 활용하는 경우에는 이들을 여러 가지 방식으로 결합할 수 있다.

제네레이터 함수를 결합하는 일반적인 방법은 합성 함수(composite function)를 만드는 경우다. (f(x) for x in range())를 계산하는 제네레이터가 있다고 가정해보자. 이때 g(f(x))를 계산하고 싶다면, 두 제네레이터를 결합하는 데는 여러 방법이 있다.

```
g_f_x = (g(f(x)) for x in range()) 
```

기술적으로 문제는 없으나 이는 재사용을 막는다. 식을 재사용하기보다는 코드를 재작성하기 때문이다.

다음과 같이 한 가지 식을 다른 식 안에서 바꿀 수도 있다.

```
g_f_x = (g(y) for y in (f(x) for x in range())) 
```

이러한 방식을 사용하면 단순한 치환으로 조합이 가능하다는 장점이 있다. 이를 재사용을 강조하는 방식으로 작성하는 방법은 다음과 같다.

```
f_x = (f(x) for x in range())
g_f_x = (g(y) for y in f_x)
```

이 방법은 최초의 식인 (f(x) for x in range())에 손을 대지 않는다는 장점이 있다. 그냥 해당 식을 변수에 대입했을 뿐이다.

그 결과로 얻은 합성 함수도 제네레이터 식이다. 따라서 이 또한 지연 계산을 수행한다. g_f_x에서 다음 값을 가져오면 f_x에서도 값을 하나 가져오고, 다시 원래의 range() 함수에서도 값을 하나 추출하게 된다는 의미다.


---